# Aula 6 - DBSCAN

Agora que já começamos a nos familiarizar com a aprendizagem não supervisionada e com o K-Means, na aula de hoje, vamos avaliar um novo método para clusterização: o **DBSCAN**.

<img src = "https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png">

https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html

____
____
_____

## DBSCAN

Veja, abaixo, uma figura com um exemplo ilustrativo da diferença de clusterização entre o K-Means e o DSBCAN. Você consegue imaginar motivos pelos quais surgem essas diferenças nos grupos de pontos no espaço de atributos?

<img src = "https://miro.medium.com/v2/resize:fit:4800/format:webp/1*rfi9uHjGPdNgXgxe9xWvVw.png">

## DBSCAN

O DBSCAN é um algoritmo de clusterização baseado no conceito de **densidade**.

O nome do algoritmo é uma sigla, que explica bem seu funcionamento: **D**ensity-**B**ased **S**patial **C**lustering of **A**pplications with **N**oise.

O algoritmo foi proposto com o objetivo de proporcionar uma técnica de clusterização que possa funcionar **mesmo quando os clusters a serem criados não forem uniformes**, tendo **tamanho, forma e densidade variáveis**. 

Além disso, por construção o método funciona bem em contextos em que há **ruídos/outliers**, sendo capaz de detectá-los sem influenciar a criação dos clusters. 

Por fim, uma vantagem enorme é o fato do algoritmo **não demandar a determinação prévia da quantidade de clusters**, o que é uma vantagem interessante se não houver indicações do problema de negócio para esta determinação (embora, como veremos, ainda há hiperparâmetros importantes a serem determinados).

Vamos entender o funcionamento do algoritmo mais a fundo!

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Princípio de funcionamento</h2>
<ul>
<li>O DBSCAN tem como princípio fundamental a <b>determinação de regiões de alta densidade de observações</b>, que são <b>separadas entre si por regiões de baixa densidade</b>.
</ul>
</span>
<br>
</div>

Lembrando que, por ser um algoritmo não-supervisionado de clusterização, quando nos referimos a "regiões" cuja densidade será aferida, estamos nos referindo a regiões **do espaço de features**.

Uma pergunta natural é: **como determinar a densidade de uma região?** Para responder a esta pergunta, precisamos de algumas definições:

> **Densidade em um ponto $P$:** número de pontos dentro de um círculo de raio $\epsilon$ centrado no ponto $P$ (região chamada de vizinhança-$\epsilon$ de $P$);

> **Região densa**: dizemos que uma região é densa se o círculo de raio $\epsilon$ contém pelo menos um número mínimo de pontos (que chamaremos de $\text{minPts}$. Uma região densa **formará um cluster**.

Para visualizar as definições acima, considere a figura a seguir:

<img src=https://www.researchgate.net/publication/315326812/figure/fig2/AS:473095908663297@1489806262333/A-cluster-consists-of-core-points-red-and-border-points-green-Core-points-have-at.png width=500>

<img src=https://www.researchgate.net/publication/335485895/figure/fig2/AS:797412515909651@1567129367940/A-single-DBSCAN-cluster-with-Core-Border-and-Noise-Points.ppm width=500>

Dada a definição acima, podemos classificar pontos dentro de um cluster como:

> **Core points (pontos centrais)**: são pontos que estão no interior dos clusters (regiões densas). Matematicamente, um ponto é considerado core **se sua densidade é de pelo menos $\text{minPts}$**, ou seja, se **há pelo menos $\text{minPts}$ pontos dentro do círculo de raio $\epsilon$ centrado no ponto**.

> **Border points (pontos de fronteira)**: são pontos que estão na fronteira de um cluster. Matematicamente, estes pontos **têm densidade menor que $\text{minPts}$**, mas que **fazem parte da vizinhança-$\epsilon$ de um ponto central**.

> **Noisy points (pontos de ruído/outliers)**: são pontos que não são centrais nem de fronteira. Estes pontos não fazem parte do cluster, e são considerados outliers.


Olhando para as definições acima, e pras figuras, fica claro que $\epsilon$ e $\text{minPts}$ são os hiperparâmetros do modelo -- e que os clusters gerados são fortemente dependentes destes hiperparâmetros!


> - $\epsilon$ (`eps` no sklearn): determina o quão próximos (relativo a uma dada **métrica de distância**) os pontos devem estar entre si para serem considerados vizinhos, e, eventualmente, parte de um cluster. Na prática, **se a distância entre dois pontos for menor ou igual a $\epsilon$, os pontos serão considerados vizinhos**;
<br><br>
>Se o valor de `eps` for muito pequeno, grande parte dos dados não serão clusterizados - muitos pontos serão considerados outliers, pois não haverá vizinhos suficientes para gerar uma região densa;<br><br>
>Por outro lado, se o valor de `eps` for muito grande, os clusters se fundirão, e a maioria dos pontos estarão em um único, grande cluster.<br><br>
>Portanto, a escolha de `eps` está muito relacionada com **a escala** das features, o que demanda cuidadosa análise exploratória.<br><br>
>Além disso, note que o  `eps` depende também fortemente da **métrica de distância** (`metric` no sklearn) a ser utilizada.


> - $\text{minPts}$ (`min_samples` no sklearn): o número mínimo de pontos que devem ser vizinhos para formar uma região densa, que será um cluster.
<br><br>
Valores maiores de `min_samples` são preferíveis para datasets com outliers, formando clusters mais significativos (isto é, um cluster só será formado se realmente tiver uma alta densidade).

Para algumas dicas práticas de como estimar bons valores para os hiper-parâmetros, [clique aqui](https://en.wikipedia.org/wiki/DBSCAN#Parameter_estimation).



-----
**Leitura complementar**

[Understanding DBSCAN and implementation with python](https://towardsdatascience.com/understanding-dbscan-and-implementation-with-python-5de75a786f9f)

__________

Agora que entendemos os princípios e principais hiperparâmetros do DBSCAN, vamos agora nos atentar um pouco melhor aos passos de execução do algoritmo.

> **Passo 1**: o algoritmo escolhe aleatoriamente um dos pontos, e sua vizinhança-$\epsilon$ é calculada;

> **Passo 2**: se este ponto tem $\text{minPts}$ em sua vizinhança-$\epsilon$, a formação do cluster é iniciada (veja próximo passo). Se não, o ponto é marcado como outlier (mas pode ser considerado como border point de um outro cluster posteriormente). Se for um outlier, volte ao passo 1;

> **Passo 3**: se o ponto for um core point, todos os pontos na vizinhança são agregados ao cluster, e o passo 1 é aplicado a cada um deles;

> **Passo 4**: o processo do passo 3 é continuado até que todos os pontos tenham um cluster associado ou esteja marcado como noise.

Podemos visualizar a seguir o DBSCAN em funcionamento:


<img src="http://data-analysis-stats.jp/wp-content/uploads/2019/09/DBSCAN_01.gif" width=400>


<img src=https://i.pinimg.com/originals/bb/3d/5e/bb3d5e522cbcb2dd07a81f8118de2041.gif width=500>


A classe do sklearn é esta: [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)


Agora, vamos avaliar duas situações, possibilitando comparações com o K-means:
- a aplicação do DBSCAN em um dataset artificial, como fizemos na aula passada;
- uma aplicação para clusterização de estações de metrô em São Paulo.

____


### DBSCAN na prática: dataset artificial

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs

Geração de dados (como fizemos na última aula)

In [ ]:
X, _ = make_blobs(n_samples = 300,
                 n_features = 2,
                 centers = 4,
                 cluster_std = 0.6,
                 random_state = 0)

plt.figure(figsize = (8,6))
plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

plt.title("Espaço de features - X1 e X2 - dos dados artificiais")
plt.show()

Como estamos lidando com distâncias, vamos normalizar as features de entrada:

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

plt.figure(figsize = (8,6))
plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

plt.title("Espaço de features - X1 e X2 - dos dados artificiais")
plt.show()

Vamos, agora, colocar os dados em um dataframe

In [ ]:
X_df = pd.DataFrame(X, columns = "x1 x2".split())
X_df

Agora, vamos utilizar o [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) nos nossos dados para encontrar clusters.

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
dbscan.fit(X) # por padrão: eps = 0.5 e min_samples = 5

In [ ]:
# Acessando os labels
dbscan.labels_

**Observação:** com o DBSCAN, dados considerados outliers/ruído (*noisy data*) são, por padrão, rotulados com $-1$.

Similarmente como fizemos para o K-means na última aula, vamos, agora, definir uma função para visualizar nossos agrupamentos para uma entrada específica dos parâmetros *eps* e *min_pts*.

In [ ]:
def plot_dbscan(X_df, eps, min_pts):
     # Instanciamento e fit do modelo
    dbscan = DBSCAN(eps = eps, min_samples = min_pts)
    dbscan.fit(X_df)

    # ========================================
    # estruturação dos resultados
    labels_clusters = dbscan.labels_
    labels_series = pd.Series(labels_clusters, name="label")

    df_result = pd.concat([X_df, labels_series], axis=1)
    n_clusters = pd.Series(labels_clusters).nunique()

    # ========================================
    print(f"DBSCAN com eps={eps} e minPts={min_pts}\nNúmero de clusters: {n_clusters}")
    sns.jointplot(data=df_result, x="x1", y="x2", hue="label", palette="viridis")
    plt.show()
    
    print("Quantidade de pontos em cada clusters:")
    print(pd.Series(labels_clusters).value_counts())
    
    return df_result, n_clusters

Vamos avaliar nossa função para os valores-padrão dos parâmetros do scikit-learn?

In [ ]:
df_result, n_clusters = plot_dbscan(X_df,
                                   eps = 0.5,
                                   min_pts = 5)

Podemos observar que, para os valores-padrão dos parâmetros de entrada, obtemos apenas um grande cluster de dados - o que não nos auxilia. Sendo assim, o que acontece quando variamos os parâmetros?

In [ ]:
df0, _ = plot_dbscan(X_df, eps = 0.1, min_pts = 5)

In [ ]:
df0.groupby('label')['label'].count()

In [ ]:
# variando o eps
eps_values = np.linspace(0.1, 0.5, 5)
n_clusters = []

for eps in eps_values:
    _, n = plot_dbscan(X_df, eps = eps, min_pts = 5)
    n_clusters.append(n)

In [ ]:
n_clusters

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(eps_values, n_clusters, marker = 'o')
plt.title('Número de clusters em função de $\epsilon$')
plt.ylabel('Número de clusters')
plt.xlabel('$\epsilon$')

Conforme esperaríamos, à medida que aumentamos $\epsilon$, conseguimos detectar menos agrupamentos com o DBSCAN (já que é necessária uma distância cada vez menor entre os pontos).

E se variarmos o mínimo de pontos?

In [ ]:
# variando o minPts
minPts_values = range(1,10)
n_clusters = []

for minPts in minPts_values:
    _, n = plot_dbscan(X_df, eps = 0.2, min_pts = minPts)
    n_clusters.append(n)

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(minPts_values, n_clusters, marker = 'o')
plt.title('Número de clusters em função do mínimo de pontos')
plt.ylabel('Número de clusters')
plt.xlabel('$minPts$')

Por fim, vamos identificar os outliers?

In [ ]:
df_result, _ = plot_dbscan(X_df, eps = 0.25, min_pts = 5)

In [ ]:
# Identificando os outliers
df_result.query("label==-1")

Se quisermos, podemos remover os outliers para visualizar nossos clusters:

In [ ]:
df_result.query("label != -1").drop('label', axis = 1)

In [ ]:
sns.jointplot(data = df_result.query("label != -1"),
             x = 'x1',
             y = 'x2',
             hue = 'label')

___